## Final Project Submission

Please fill out:
* Student name: 
* Student pace: self paced / part time / full time
* Scheduled project review date/time: 
* Instructor name: 
* Blog post URL:


# This project was an exercise in multiple linear regression, using the Housing dataset from King County. The process follows the OSEM-I Data Science work flow.

In [ ]:
#O: Obtain Data

In [ ]:
##The first step is to upload the data and load the numpy/pandas libraries, then take a 
##look at what the dataframe looks like, inspect column names

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
from scipy import stats
from statsmodels.formula.api import ols
import warnings
warnings.filterwarnings('ignore')
kchouse = pd.read_csv('data/kc_house_data.csv')
kchouse.head()

In [ ]:
kchouse.info()

In [ ]:
kc_new = kchouse.drop(['date', 'view', 'sqft_above', 'sqft_basement', 'yr_renovated', 'zipcode', 'lat', 'long', 'sqft_living15', 'sqft_lot15'], axis=1)
kc_new.info()

In [ ]:
#I created a correlation heatmap to ID any potential independent variables that
#exhibit multicolinearity, as that would violate one of the necessary assumptions
#we need for Linear Regression

In [ ]:
kc_new.isna().value_counts()

In [ ]:
kc_new['waterfront'] = kc_new['waterfront'].fillna(0.0)
kc_new.isna().value_counts()

In [ ]:
#I wanted to make sure there was one entry per ID, and drop any double-entries

In [ ]:
kc_new['id'].value_counts()

In [ ]:
kc_new.drop_duplicates(subset = 'id', inplace = True)

In [ ]:
kc_new.info()

In [ ]:
#The next steps, Scrub and Explore, do mean separate things, but tend go hand in 
#hand. As we Explore more representations of our data, we gain more particular 
#insight into its characteristics. 

In [ ]:
#I noticed there were null values in the waterfront column. The values 1 and 0 
#represent if the property is on a waterfront or not. I'm making and educated 
#assumption that that means there is no waterfront, so I changed the null values 
#to a 0.

In [ ]:
kc_new.describe()

In [ ]:
#some notes: # Mean price is $540,296.57 
             # std $367368.14
             # min $78,000.0 
             # 25% $322,000.0
             # 50% $450,000.0,
             # 75% $645000.0
             # max $7,700,000.0

In [ ]:
### big ol scatter matrix to help discern between continuous/categorical variables

In [ ]:
pd.plotting.scatter_matrix(kc_new, figsize=[12,12])

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
corr = kc_new.corr()
mask = np.zeros_like(corr)
mask[np.triu_indices_from(mask)] = True

with sns.axes_style("white"):

    f, ax = plt.subplots(figsize=(14, 12))

    ax = sns.heatmap(corr, mask=mask, square=True, annot = True, cmap = 'coolwarm')

In [ ]:
df = kc_new.corr().abs().stack().reset_index().sort_values(0, ascending=False)

# zip the variable name columns (Which were only named level_0 and level_1 by default) in a new column named "pairs"
df['pairs'] = list(zip(df.level_0, df.level_1))

# set index to pairs
df.set_index(['pairs'], inplace = True)

#d rop level columns
df.drop(columns=['level_1', 'level_0'], inplace = True)

# rename correlation column as cc rather than 0
df.columns = ['cc']

# drop duplicates. This could be dangerous if you have variables perfectly correlated with variables other than themselves.
# for the sake of exercise, kept it in.
df.drop_duplicates()
df[(df.cc>.75) & (df.cc <1)]

In [ ]:
def hist_and_scatter(feature, title, featurename):
    figure, ax = plt.subplots(1, 2, figsize=(8, 6))
    ax[0].hist(feature)
    ax[1].scatter(feature, kc_new['price'])
    ax[0].set_xlabel(featurename)
    ax[1].set_xlabel(featurename)
    ax[1].set_ylabel('price')
    figure.suptitle(title)
    plt.tight_layout(rect=[0, 0.03, 1, 0.9])
    plt.show()

In [ ]:
for col in kc_new.columns:
    hist_and_scatter(
        kc_new[col], f'Distribution and Relationship of Sale Price and {col}', col)

In [ ]:
# Get variable correlations
corrs = kc_new.corr()

# Show variables with the highest relationship to SalePrice
corrs.abs().sort_values('price', ascending=False)['price']

In [ ]:
# The variables with the clearest linear relationship to price are sqft_living,
# grade, and # bathrooms. Bedrooms/Presence of a Waterfront, and Floors
# have a weak positive correlation 

## OLS with most correlated feature ##

In [ ]:
X = kc_new['sqft_living'] 
Y = kc_new['price']

X = sm.add_constant(X)

model = sm.OLS(Y, X).fit()
predictions = model.predict(X) 

model.summary()

In [ ]:
X = kc_new.drop(['price'], axis = 1)
y = kc_new['price']
X = sm.add_constant(X) 

model = sm.OLS(y, X).fit()
predictions = model.predict(X) 

model.summary()

In [ ]:
fig = sm.graphics.qqplot(model.resid, dist=stats.norm, line='45', fit=True)

# Are we meeting our assumptions for MLS? #
## Multicolinearity, Homoscedascity, Normality ##

# Preprocessing #

In [ ]:
####: drop id & yrbuilt
### .608 - .611 (ols)
### trying to drop bathrooms as well to address potential mc
### .608, basically no difference
### drop just id and use yr_built as numeric
### .655, qq plot isn't greatest
##### id and yr_built are very gone
### drop id, yr_built, sqft_lot, 

In [ ]:
continuous = ['sqft_living', 'sqft_lot']
categoricals = ['waterfront', 'bedrooms','grade', 'floors', 'bathrooms', 'condition']
kccat = kc_new[categoricals]
kccon = kc_new[continuous]


In [ ]:
condition_ohe = pd.get_dummies(kccat['condition'], prefix = 'condition', drop_first=True)
waterfront_ohe =pd.get_dummies(kccat['waterfront'], prefix = 'waterfront', drop_first=True)
grade_ohe= pd.get_dummies(kccat['grade'], prefix = 'grade', drop_first=True)
bed_ohe= pd.get_dummies(kccat['bedrooms'], prefix = 'bed', drop_first=True)
bath_ohe= pd.get_dummies(kccat['bathrooms'], prefix = 'bath', drop_first=True)
floors_ohe= pd.get_dummies(kccat['floors'], prefix = 'floors', drop_first=True)


In [ ]:
ohe_concat = pd.concat([waterfront_ohe, bed_ohe, floors_ohe, grade_ohe, bath_ohe, condition_ohe], axis = 1)
ohe_concat

In [ ]:
price_log = np.log(kc_new['price'])
def normalize(feature):
    return (feature - feature.mean()) / feature.std()

price_log_norm = normalize(price_log)
price_log_norm

In [ ]:
# log and normalize features
log_names = [f'{column}_log' for column in kccon.columns]

kccon_log = np.log(kccon)
kccon_log.columns = log_names

log_names

In [ ]:
def normalize(feature):
    return (feature - feature.mean()) / feature.std()

kc_log_norm = kccon_log.apply(normalize)

In [ ]:
preprocessed = pd.concat([ohe_concat, kc_log_norm, price_log_norm], axis = 1)

In [ ]:
preprocessed.columns = preprocessed.columns.str.replace('.','_')

In [ ]:
preprocessed.info()

In [ ]:
X = preprocessed.drop(['price'], axis = 1)
y = preprocessed['price']
X = sm.add_constant(X) # adding a constant

model = sm.OLS(y, X).fit()
predictions = model.predict(X) 

model.summary()

## Baseline Train Model ##

In [ ]:
X = preprocessed.drop(['price'], axis = 1)
y = preprocessed['price']
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y)
print(len(X_train), len(X_test), len(y_train), len(y_test))

In [ ]:
X = X_train
y = y_train
X = sm.add_constant(X) # adding a constant

model = sm.OLS(y, X).fit()
predictions = model.predict(X) 

model.summary()

In [ ]:
from sklearn.linear_model import LinearRegression
linreg = LinearRegression()
linreg.fit(X_train, y_train)
preds = linreg.predict(X_train)

In [ ]:
LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [ ]:
linreg.coef_

In [ ]:
linreg.intercept_

In [ ]:
from sklearn.metrics import r2_score
r2_score(y_train, preds)

In [ ]:
df = X_train.corr().abs().stack().reset_index().sort_values(0, ascending=False)

# zip the variable name columns (Which were only named level_0 and level_1 by default) in a new column named "pairs"
df['pairs'] = list(zip(df.level_0, df.level_1))

# set index to pairs
df.set_index(['pairs'], inplace = True)

#d rop level columns
df.drop(columns=['level_1', 'level_0'], inplace = True)

# rename correlation column as cc rather than 0
df.columns = ['cc']

# drop duplicates. This could be dangerous if you have variables perfectly correlated with variables other than themselves.
# for the sake of exercise, kept it in.
df.drop_duplicates()
df[(df.cc>.75) & (df.cc <1)]

In [ ]:

def stepwise_selection(X, y, 
                       initial_list=[], 
                       threshold_in=0.01, 
                       threshold_out = 0.05, 
                       verbose=True):
    """ 
    Perform a forward-backward feature selection 
    based on p-value from statsmodels.api.OLS
    Arguments:
        X - pandas.DataFrame with candidate features
        y - list-like with the target
        initial_list - list of features to start with (column names of X)
        threshold_in - include a feature if its p-value < threshold_in
        threshold_out - exclude a feature if its p-value > threshold_out
        verbose - whether to print the sequence of inclusions and exclusions
    Returns: list of selected features 
    Always set threshold_in < threshold_out to avoid infinite looping.
    See https://en.wikipedia.org/wiki/Stepwise_regression for the details
    """
    included = list(initial_list)
    while True:
        changed=False
        # forward step
        excluded = list(set(X.columns)-set(included))
        new_pval = pd.Series(index=excluded, dtype='float64')
        for new_column in excluded:
            model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included+[new_column]]))).fit()
            new_pval[new_column] = model.pvalues[new_column]
        best_pval = new_pval.min()
        if best_pval < threshold_in:
            best_feature = new_pval.idxmin()
            included.append(best_feature)
            changed=True
            if verbose:
                print('Add  {:30} with p-value {:.6}'.format(best_feature, best_pval))

        # backward step
        model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included]))).fit()
        # use all coefs except intercept
        pvalues = model.pvalues.iloc[1:]
        worst_pval = pvalues.max() # null if pvalues is empty
        if worst_pval > threshold_out:
            changed=True
            worst_feature = pvalues.idxmax()
            included.remove(worst_feature)
            if verbose:
                print('Drop {:30} with p-value {:.6}'.format(worst_feature, worst_pval))
        if not changed:
            break
    return included

In [ ]:
result = stepwise_selection(X_train, y_train, verbose = True)
print('resulting features:')
print(result)

In [ ]:
X_fin = X_train[result]
X_with_intercept = sm.add_constant(X_fin)
model = sm.OLS(y_train,X_with_intercept).fit()
model.summary()

In [ ]:
fig = sm.graphics.qqplot(model.resid, dist=stats.norm, line='45', fit=True)

# Ranked Feature Selection with Scikit #

In [ ]:
from sklearn.feature_selection import RFE

linreg = LinearRegression()
selector = RFE(linreg, n_features_to_select = 25)
selector = selector.fit(X_fin, y_train.values.ravel()) # convert y to31d np array to prevent DataConversionWarning
selector.support_ 

In [ ]:
selected_columns = X_fin.columns[selector.support_ ]
linreg.fit(X_fin[selected_columns],y_train)

In [ ]:
yhat = linreg.predict(X_fin[selected_columns])

In [ ]:
SS_Residual = np.sum((y-yhat)**2)
SS_Total = np.sum((y-np.mean(y))**2)
r_squared = 1 - (float(SS_Residual))/SS_Total
adjusted_r_squared = 1 - (1-r_squared)*(len(y)-1)/(len(y)-X_fin[selected_columns].shape[1]-1)

In [ ]:
r_squared

In [ ]:
adjusted_r_squared

In [ ]:
import matplotlib.pyplot as plt

residuals = y - preds

plt.scatter(preds, residuals)
plt.hlines(0, preds.min(), preds.max())
